<h1>Prepare your dataset for PETS2.0</h1>

PETS 2.0 will take care of processing the electron diffraction data, finding the cell parameters, and extracting the hkl dataset that is required for structure solution software such as ShelXL. However, PETS 2.0 works with '.tif' files not '.mrc' fiels and requires an instruction files to provide a list of file names and some other information. The purpose of this script is to write the instruction file for PETS 2.0 and, if needed, to convert the '.mrc' files to the 16 bit '.tif' files.

There are four main parts to this script:
1. Import libraries and set working directory
2. Write instruction file for PETS 2.0
3. Read the header of the mrc file
4. Drop spaces from file name
5. Convert images to unsigned 16 bits tif

Part 1 needs to be run after opening the python script as it loads the libraries used by the parts that follow and set some general settings for the appearance of plots.

Part 2 will ask for the working directory and creates the instruction file for PETS 2.0. 
Part 3 and 4 are only needed if you wish to read all the file metadata ar remove spaces from a file name, respectively. 
Part 4 converts the .mrc files to a 16 bit tif that is readable by PETS 2.0 and has sufficient dynamic range for the peak finding routine.



<h1>Import libraries and set working directory</h1>

The code in the box below will import all needed python libraries. It only needs to be run once at the beginning of the session. To run, click anywhere in the code box and then click play. When the code finished to run, the gray brackets at the left of the box will display a number.

In [1]:
# Import Libraries
#Import the libraries for the sripts. This import includes all the libraries needed for all the scripts in this document.
import mrcfile
import numpy as np
import os
import csv
import matplotlib.pyplot as plt
from PIL import Image

#Set the default style for plots - make changes here if you want a different font or font size
from matplotlib import rcParams
rcParams['font.family'] = 'Arial'
rcParams['font.size'] = 16

The first step, after importing the libraries is to set the working directory where the microED files are stored by running the code in the box below:

In [2]:
## Select Directory
# Ask the user to input a directory path
directory = input("Please enter the directory path: ")
# Verify if the path is valid
if os.path.isdir(directory):
    os.chdir(directory)
else:
    print("Invalid directory. Please try again.")


Please enter the directory path:  C:\Users\debora_berti\Documents\epud_Joel_set1\JA-OpADiol2-particle22


Next, compile a list of file names, find the alpha angle of each file, and also find the resolution in terms of pixel size:

<h1>Write instruction file for PETS 2.0</h1>
Next, write the instruction file that PETS 2.0 uses to set up the processing session. This file contains a file name list, pixel resolution, electron wavelength, and other information relevant to processing. 

In [ ]:
### Find Information 
#variable to hold the file names
fcount = []
name = []

#######################

#go through the files with the mrc extension and write their name into the fcount variable.
for file in os.listdir(directory):
    if file.endswith('.mrc'):
        fcount.append(file)
        name.append(os.path.splitext(os.path.basename(file))[0])

#######################
#Print one file name to verify that the script worked
#print(fcount[0])
#print(name[1])
 ######################
#Read the alpha angle
#variable to hold the alpha angle

a = []

#loop through the files in fcount, extract the alpha angle and write it in the variable a

for fname in fcount:
    mrc = mrcfile.open(fname)
    head = mrc.indexed_extended_header
    head.tofile('headerInfo.txt',',')
    with open('headerInfo.txt', 'r') as file:
        contents = file.read()
        hdlist = contents.split(',')
        a.append(hdlist[10][:8])
        
    mrc.close()
print(a[0])

mrc = mrcfile.open(fname)
#############
#Find the pixel size in the heaer

PixelSize = "The pixel dimensions along x, y, and z in reciprocal microns is: {}"

print(PixelSize.format(mrc.voxel_size))

mrc.close()


#####################################

####Instruction File
#Function to count the number of files with extension '.mrc' in the current directory:
def count_files_by_extension(directory_path, extension):
    """Count the number of files with a specific extension."""
    return sum(1 for file in os.listdir(directory_path) 
               if file.endswith(extension) and os.path.isfile(os.path.join(directory_path, file)))
########################
#Count the total numbre of '.mrc' files in the directory
tot_files = count_files_by_extension(directory, ".mrc")
tot = tot_files -1
############

#Open a file for writing:
f = open(name[0] + ".pts", "a")
    
f.write("lambda 0.0196\n")
f.write("Aperpixel 0.001869\n")
f.write("omega 5.0\n")
f.write("phi 0.0\n")
f.write("noiseparameters 3.5 38\n")
f.write("reflectionsize 10\n")
f.write("bin 2\n")
f.write("imagelist\n")




#Write the file name list and the alpha, beta angles
#Create a counter variable
j = 0

for x in range(0,tot_files):
    text = "{}.tif {}   0.00"
    
    f.write(text.format(name[x],a[j]) + "\n")
    j = j + 1

f.write("endimagelist\n")
f.write("################################################\n")
f.write("#	MEASUREMENT CONDITIONS\n")
f.write("#  Start series = 	    -75 deg\n")
f.write("#  Stop series = 	     70 deg\n")
f.write("#  Tilt step = 		      0.5 deg\n")
f.write("#  Camera Length = 	      430 mm\n")
f.write("#  DP exposure = 	      500 ms\n")
f.write("#  CL aperture = 	        50 um\n")
f.write("#  SA aperture = 	        100 um\n")
f.write("#  Spot Size = 		         6\n")
f.write("#  GL =                         5 \n")
f.write("#  Magnification = 	     11000\n")
f.write("################################################\n")
f.close()

<h1> Read the header of the mrc file</h1>
The next code snippet allows you to inspect all the information contained in the extended header of an mrc file in the working directory. Run this code if you wish to have more information on optics and acquisition parameters.

In [ ]:
##### Read Header 
# Ask the user to input the name of a file in the working directory to print its full extended header:
fnameh = input("Please enter the file name: ")

#Validate that the name entered refers to an existing file in the directory:

if os.path.isfile(fnameh):
    mrc2 = mrcfile.open(fnameh)
    head2 = mrc2.indexed_extended_header
    print(head2)
else:
    print("Invalid file name. Please try again.")



<h1> Drop spaces from file name </h1>
PETS 2.0 does not accept file names containing spaces. Ifyou have spaces in the file names, you can run this code to remove them.

In [ ]:
###### Eliminate Spaces 
# Function to remove spaces from filenames
def remove_spaces_in_filenames(directory):
    try:
        # Loop through all files in the directory
        for filename in os.listdir(directory):
            # Check if the filename contains spaces
            if ' ' in filename:
                # Replace spaces with no space (or you can use an underscore if preferred)
                new_filename = filename.replace(' ', '')
                old_file = os.path.join(directory, filename)
                new_file = os.path.join(directory, new_filename)
                # Rename the file
                os.rename(old_file, new_file)
                print(f'Renamed: {old_file} -> {new_file}')
    except Exception as e:
        print(f"Error: {e}")

# Directory containing the files
directory_path = "C:/Users/debora_berti/Documents/epud_Joel_set1/My First Aggregate/Process_PETS"

# Call the function to remove spaces from filenames
remove_spaces_in_filenames(directory_path)

<h1> Convert images to unsigned 16 bits</h1>

In some cases, you may need to convert the bit depth of your images to a signed 16 bits tif format which works best with PETS 2.0.

The code below helps you by correctly handling the reduction in the number of intensity levels that is required to degrade from 32 bits to 16 bits. While 32 bits contains excessive information, 8 bits (256 levels of gray) may not be sufficient to describe the dynamic ranges from the intense direct beam to weak diffraction spots.

In [ ]:
####### Convert to Tif

#Code to convert mrc images to an unsigned 16 bit tif



#Open the mrc file and lad the data into a variable:
for fid in fcount:
    imgRaw = mrcfile.open(fid, permissive=True)
    img = imgRaw.data.astype(np.float32)

#Ensure tha image is within the 16 bit range for visualization, without this code the negative range of the grayscale will be clipped:

   # min_val = img.min()
   # if min_val < 0:
   #     pos_img = img - min_val
   # else:
   #     pos_img = img
        
    #img = (-pos_img * 65535).astype(np.uint16)
  
    
    
    ## Alternative way to adjust the contrast to prevent clipping
    min_val = img.min()
    max_val = img.max()
    
    scaled_img = (img - min_val) / (max_val - min_val )
    img = (scaled_img * 65536).astype(np.uint16)

    imgRaw.close()
    

#Convert Numpy array to a PIL image then save it:
    img = Image.fromarray(img)
    img.save(os.path.splitext(os.path.basename(fid))[0] +'.tif', format = 'TIFF')




<h1> The End</h1>
That's all! You are ready to start your PETS 2.0 session.